## How To Run

### Requirements

#### Python3

install `python3`

```sh
# virtual environment is recommended (eg. venv, virtualenv, conda, ...)
pip install -r requirement.txt
```

#### Openslide

##### Windows

* download `openslide-bin` from [openslide](https://openslide.org/download/#binaries)
* unzip and rename to `openslide-bin` (remove the version in name)
* move `openslide-bin` into project directory

> run Environment Setup before any other notebook cell

##### Other

install `openslide` within your distro

### Dataset

* slides under `./dataset/ndpi/`
* labels under `./dataset/`

### Code

* Find Cells :13 for `patch.csv` and `mass.csv` 
* Find Cells :15 ~ :21 for filter
* Find Cells :47 for parallel method
* Visualize Mass :17 for `patch.csv` and `mass.csv` 
* Visualize Cell :24 for `patch.csv` and `mass.csv` 

In [5]:
# Environment Setup (for windows)

import sys
import os
from pathlib import Path

ROOT_DIR = Path('../')
if sys.platform == "win32":
    os.add_dll_directory(ROOT_DIR / "openslide-bin" / "bin")

In [ ]:
# Parse Labels (output mass.csv)

from pathlib import Path

import utils
import dataset

ROOT_DIR = Path('../')
utils.set_log_lvl('WARN')

labels = utils.load_json(ROOT_DIR / 'dataset' /
                         'Thyroid_AI_data_20220103.json')
mass_df = dataset.get_mass_info(labels)
mass_df.to_csv(ROOT_DIR / 'output' / 'mass.csv')

mass_df.info()

In [ ]:
# Slide Info (output slide_info.csv)

from pathlib import Path

import utils
import dataset

ROOT_DIR = Path('../')
utils.set_log_lvl('WARN')

path_list = list((ROOT_DIR / 'dataset' / 'ndpi').glob('*.ndpi'))
slide_df = dataset.get_slide_info(path_list)
slide_df.to_csv(ROOT_DIR / 'output' / 'slide_info.csv')

slide_df.info()

In [ ]:
# Generate Patch (output patch.csv)

import openslide
import pandas as pd
from pathlib import Path

import utils

ROOT_DIR = Path('../')

patch = []
for ndpi in (ROOT_DIR / 'dataset' / 'ndpi').glob('*.ndpi'):
    slide = openslide.open_slide(ndpi)
    name = ndpi.stem
    (w, h) = slide.dimensions
    for x_off in range(0, w, 2048):
        for y_off in range(0, h, 2048):
            pw = min(2048, w - x_off)
            ph = min(2048, h - y_off)
            uid = utils.mass_hash(name, (x_off, y_off, pw, ph))
            patch.append([uid, name, 'patch', x_off, y_off, pw, ph])
    # break

patch = pd.DataFrame(
    patch,
    columns=["uid", "slide", "tag", "x_min",
             "y_min", "width", "height"],  # type: ignore
)
patch.set_index("uid", inplace=True)
if patch.index.has_duplicates:
    raise

patch.to_csv(ROOT_DIR / 'output' / 'patch.csv')
patch.info()

In [ ]:
# Find Cells (output cell.csv)

import pandas
from pathlib import Path

import utils
from find_cell import find_cell

ROOT_DIR = Path('../')

slide_df = pandas.read_csv(
    ROOT_DIR / 'output' / 'slide_info.csv', index_col=0)
mass_df = pandas.read_csv(ROOT_DIR / 'output' / 'patch.csv', index_col=0)
# no source
# mass_df = mass_df[mass_df['slide'].isin(slide_df.index)]
# blocklist
# mass_df = mass_df[~mass_df['slide'].isin(['PTC_20_088_P'])]
# filter tag
# mass_df = mass_df[mass_df['tag'].isin(['PTC', 'BFC'])]
# sample
# mass_df = mass_df.sample(1024, random_state=0xdeadbeef)

reader = None


def job(args):
    global reader
    (uid, slide, bbox, info) = args
    result = find_cell(slide, bbox, reader, info)  # type: ignore
    result = [(uid,) + item for item in result]
    return result


def pack_args(row):
    (uid, slide, _, x_min, y_min, width, height) = row
    return (uid, slide, (x_min, y_min, width, height), slide_df.loc[slide].to_dict())


def initializer(folder):
    global reader
    utils.set_log_lvl('WARN')
    reader = utils.SlideReader(folder)


args = [pack_args(row) for row in mass_df.itertuples()]
result = utils.parallel_map(
    True, job, args, initializer, (ROOT_DIR / 'dataset' / 'ndpi',))
result = [item for sublist in result for item in sublist]

cell_df = pandas.DataFrame(
    result, columns=['mass_uid', 'cx', 'cy', 'width', 'height', 'angle', 'score'])
cell_df.to_csv(ROOT_DIR / 'output' / 'cell.csv')
cell_df.info()

In [ ]:
# Visualize Mass

import cv2
import numpy
import pandas
import concurrent.futures
from tqdm.auto import tqdm
from pathlib import Path
from PIL import Image

import utils

ROOT_DIR = Path('../')
utils.set_log_lvl('WARN')

cell_df = pandas.read_csv(ROOT_DIR / 'output' / 'cell.csv', index_col=0)
mass_df = pandas.read_csv(ROOT_DIR / 'output' / 'mass.csv', index_col=0)
reader = utils.SlideReader(ROOT_DIR / 'dataset' / 'ndpi')


def read_mass(data):
    mass_uid, cells = data
    mass = mass_df.loc[mass_uid]
    image = reader.read_bbox(
        mass["slide"], utils.get_bbox(mass))  # type: ignore
    image = cv2.cvtColor(numpy.array(image), cv2.COLOR_RGBA2BGR)

    for cell in cells.itertuples(index=False):
        box = ((cell.cx, cell.cy), (cell.width, cell.height), cell.angle)
        hue = round(180 * cell.score)  # type: ignore
        image = cv2.ellipse(image, box=box, color=(  # type: ignore
            75, 75 + hue, 255 - hue))

    return Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))


mass_iter = cell_df.groupby('mass_uid')
with concurrent.futures.ThreadPoolExecutor() as executor:
    for mass in tqdm(executor.map(read_mass, mass_iter), total=len(mass_iter)):
        display(mass)

In [ ]:
# Visualize Cell

import numpy
import math
import pandas
import concurrent.futures
from tqdm.auto import tqdm
from pathlib import Path
from PIL import Image

import utils

ROOT_DIR = Path('../')
utils.set_log_lvl('WARN')

cell_df = pandas.read_csv(ROOT_DIR / 'output' /
                          'cell.csv', index_col=0)
# filter score
cell_df = cell_df[cell_df['score'] > 0.3]
cell_df = cell_df.sample(1600 * 16, random_state=0xdeadbeef)
# sort score
cell_df = cell_df.sort_values('score')

mass_df = pandas.read_csv(ROOT_DIR / 'output' / 'patch.csv', index_col=0)
reader = utils.SlideReader(ROOT_DIR / 'dataset' / 'ndpi')
buf_display = utils.BufferedDisplay(40, 40, 50, 50)


def read_cell(cell):
    mass = mass_df.loc[cell.mass_uid]

    off_x = math.floor(cell.cx - 60)  # type: ignore
    off_y = math.floor(cell.cy - 60)  # type: ignore
    cx = cell.cx - off_x
    cy = cell.cy - off_y
    cw = math.ceil(cell.width * 1.4)  # type: ignore
    ch = math.ceil(cell.height * 1.4)  # type: ignore

    image = reader.read_bbox(
        # type: ignore
        mass["slide"], (mass["x_min"] + off_x, mass["y_min"] + off_y, 120, 120))
    image = numpy.array(image.convert("RGB"))

    cell_image = utils.crop_rotated_rectangle(
        image, (cx, cy), (cw, ch), cell.angle)  # type: ignore

    return Image.fromarray(cell_image)


with concurrent.futures.ThreadPoolExecutor() as executor:
    for cell in tqdm(executor.map(read_cell, cell_df.itertuples(index=False)), total=len(cell_df)):
        buf_display.display(cell)

buf_display.flush()

In [ ]:
# Describe Score

import pandas
from pathlib import Path

ROOT_DIR = Path('../')
cell_df = pandas.read_csv(ROOT_DIR / 'output' /
                          'cell.csv', index_col=0)
cell_df['score'].describe()
# print(len(cell_df[cell_df['score'] >= 0.75]) / len(cell_df))